In [1]:
import flask
from flask import request
import tempfile
import requests
import whisper
import os
from twilio.rest import Client
from dotenv import load_dotenv

In [2]:
account_sid = 'AC3af69dd7bb05709d20c9e1f464c3963c'
auth_token = '247027c60ac66f6383c3b7f4ceb1f5cf'
client = Client(account_sid, auth_token)

In [8]:
message = client.messages.create(
    from_='whatsapp:+14155238886',
    body='Your appointment is coming up on July 21 at 3PM',
    to='whatsapp:+254747929113'
)

print(message.sid)

SM3a548272e5f34795e1fa0be14ff4c13a


In [4]:
client = Client(account_sid, auth_token)
app = flask.Flask(__name__)

In [6]:
from audio_inference import audiototext, texttoaudio

def transcribe(audio_url):
    response = requests.get(audio_url)
    with tempfile.TemporaryDirectory() as temp_dir:
        audio_file_path = f"{temp_dir}/audio.tmp"
        with open(audio_file_path, "wb") as audio_file:
            audio_file.write(response.content)
        result = audiototext(audio_file_path)
    return result

In [9]:
def send_message(senderId, message):
    """Send a message to the specified phone number.

    Args:
        senderId (str): Phone number to send the message to.
        message (str): Message to send.

    Returns:
        twilio.rest.Message: Result of the message creation request.
    """
    res = client.messages.create(body=message, from_ = 'whatsapp:+14155238886', to=f"whatsapp:+{senderId}")
    return res

In [10]:
@app.route("/")
@app.route("/home")
def home():
    return "Hello Bot"

In [11]:
@app.route("/whatsapp", methods=["GET", "POST"])
def whatsapp():
    """Endpoint for handling incoming WhatsApp messages."""

    senderId = request.form["From"].split("+")[1]
    mediaUrl = request.form["MediaUrl0"]

    try:
        response_message = transcribe(mediaUrl)
        send_message(senderId=senderId, message=response_message)
    except Exception as e:
        print(e)
    return "200"

In [ ]:
if __name__ == "__main__":
    app.run(port=5000, debug=True)